In [3]:
import spacy    
import unicodedata
import re

In [4]:
# Loads NLP English model
nlp = spacy.load('en')

In [108]:
processed = False
paper = "data/texts/ebola/10.1007/s00705-020-04768-3.txt" # oliver test paper
#paper = 'random_full.txt'

# Reads text
if 1:
    with open(paper, 'r') as file:
        text = file.read().replace('\n', ' ')
else:
    text = "In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus. The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88%2,5,25,38,43,49,50 (Table 3 (available online only)). The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69%1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42%19,46,47 (Table 5). For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70%, and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38. The CFR among EVD cases reported by Nigeria (n=20) was 40%54). A second, unrelated EVD outbreak occurred in Équateur province, DRC between July and October 2014 resulting in 69 confirmed and probable cases with a CFR of 74%49. The CFR for Marburg is approximately 80%55–57)."
    
text = unicodedata.normalize("NFKD", text) # replaces \xa0 with " "
text = re.sub(r'(?<=[.]) (?=[^\s])', r' ', text) # spaces out concatenated sentences
text

"Potential strategies for combating COVID-19  Abstract Coronavirus disease 2019, also known as COVID-19, is caused by a novel coronavirus named severe acute respiratory syndrome coronavirus 2, or SARS-CoV-2. The infection has now catapulted into a full-blown pandemic across the world, which has affected more than 2 million people and has led to approximately 150,000 fatalities all over the world (WHO). In this review, we elaborate all currently available data that shed light on possible methods for treatment of COVID-19, such as antiviral drugs, corticosteroids, convalescent plasma, and potentially effective vaccines. Additionally, ongoing and discontinued clinical trials that have been carried out for validating probable treatments for COVID-19 are discussed. The review also elaborates the prospective approach and the possible advantages of using convalescent plasma and stem cells for the improvement of clinical symptoms and meeting the demand for an instantaneous cure.  Introduction 

In [110]:
def processText(text):
    global processed
    if not processed:
        processed = True
        text = text.replace('%','% ').replace('–',' – ').replace('-',' – ').replace('=',' = ').replace(',',', ')
        text = re.sub("[\[].*?[\]]", "", text)
        string1_protected = re.sub(r"(\d)\.(\d)", r"\1[PROTECTED_DOT]\2", text)  
        # now split (and remove empty lines)
        lines_protected = [line + "." for line in string1_protected.split(".") if line]   
        # now re-replace all "[PROTECTED_DOT]"s
        lines = [line.replace("[PROTECTED_DOT]", ".") for line in lines_protected]
        text = ' '.join(lines)
        text = re.sub(r'\s([?.!"](?:\s|$))', r'\1', text)
        text = re.sub(' +', ' ',text)
        return text
    else:
        print("Already processed")
        return text
text = processText(text)
text

Already processed


"Potential strategies for combating COVID – 19 Abstract Coronavirus disease 2019, also known as COVID – 19, is caused by a novel coronavirus named severe acute respiratory syndrome coronavirus 2, or SARS – CoV – 2. The infection has now catapulted into a full – blown pandemic across the world, which has affected more than 2 million people and has led to approximately 150, 000 fatalities all over the world (WHO). In this review, we elaborate all currently available data that shed light on possible methods for treatment of COVID – 19, such as antiviral drugs, corticosteroids, convalescent plasma, and potentially effective vaccines. Additionally, ongoing and discontinued clinical trials that have been carried out for validating probable treatments for COVID – 19 are discussed. The review also elaborates the prospective approach and the possible advantages of using convalescent plasma and stem cells for the improvement of clinical symptoms and meeting the demand for an instantaneous cure. 

In [111]:
# Convert text into NLP object
textdoc = nlp(text)

In [112]:
# Named Entity Recognition
from spacy import displacy 
displacy.render(textdoc, style='ent')

In [113]:
sents = list(str(sent) for sent in list(textdoc.sents))

import pandas as pd
pd.set_option('display.max_colwidth', None)

# Put all sentences in the dataframe
sentsdf = pd.DataFrame(sents, columns = ['Sentence'])
sentsdf

,Sentence
0,"Potential strategies for combating COVID – 19 Abstract Coronavirus disease 2019, also known as COVID – 19, is caused by a novel coronavirus named severe acute respiratory syndrome coronavirus 2, or SARS – CoV – 2."
1,"The infection has now catapulted into a full – blown pandemic across the world, which has affected more than 2 million people and has led to approximately 150, 000 fatalities all over the world (WHO)."
2,"In this review, we elaborate all currently available data that shed light on possible methods for treatment of COVID – 19, such as antiviral drugs, corticosteroids, convalescent plasma, and potentially effective vaccines."
3,"Additionally, ongoing and discontinued clinical trials that have been carried out for validating probable treatments for COVID – 19 are discussed."
4,The review also elaborates the prospective approach and the possible advantages of using convalescent plasma and stem cells for the improvement of clinical symptoms and meeting the demand for an instantaneous cure.
...,...
469,"Although extensive research is being conducted to develop a suitable vaccine against COVID – 19, there is a dire need to shift a major part of the ongoing research towards the treatment of pneumonia in patients, which is often fatal."
470,The astounding data regarding mesenchymal stem cells offers a hopeful approach to the use of an endogenous pathway for the treatment of disease.
471,"In the case of pneumonia caused by COVID – 19, the balance between the antiviral response and the regulation of LIF action against a cytokine storm may be lost, which can reduce the overall beneficial effect of therapy."
472,"Nevertheless, the study findings indicating the safe and effective use of mesenchymal stem cell therapy in treating COVID – 19 are indeed remarkable and present a modern approach to the efficient and safe treatment of critically ill patients."


In [114]:
# Listing keywords
numeric_keywords = ['distribution*','time','number*','ratio','proportion','period','±','total*','estimate*','%','review','parameter*','mean','period','value']
specific_keywords = ['CFR','case-fatality','r0','reproduct*','infect*','infections','death*','transmis*','laten*','contact','infectious','incubat*','casualties','mortal*','morbid*','outbreak*','epideme*']
contextual_keywords = ['GPE','DATE','TIME','PRODUCT']
statistical_keywords = ['fatalities', 'deaths', 'cases']

# Creating a regular expression using keywords for searching and filtering 
numeric_regex = '|'.join(numeric_keywords)
specific_regex = '|'.join(specific_keywords)
statistical_regex = '|'.join(specific_keywords)

trait_keywords = numeric_keywords + specific_keywords
trait_regex = '|'.join(trait_keywords)
trait_regex

numReg = re.compile(numeric_regex)
specReg = re.compile(specific_regex)
traitReg = re.compile(trait_regex)
statReg = re.compile(statistical_regex)

In [115]:
import re

'''
Parses a sentence, looking for keywords.

args: sentence - string of sentence.
return: specific_keyword_count - total number of specific keywords in sentence
        numeric_keyword_count - total number of numeric keywords in sentence
        contextual_keyword_count - total number of contextual entity labels in sentence
        cardinality - total number of numeric entity labels in sentence
'''
def countKeywords(sentence):
    specific_keyword_count = numeric_keyword_count = contextual_keyword_count = cardinality = 0
    
    # counts specific and numeric keywords first
    for word in sentence.split():
        if re.match(specific_regex, word): specific_keyword_count += 1
        elif re.match(numeric_regex, word): numeric_keyword_count += 1
    
    # counts key entity labels
    sentence_obj = nlp(sentence)
    ent_labels = [ent.label_ for ent in sentence_obj.ents]
    for label in ent_labels:
        if label in contextual_keywords: contextual_keyword_count += 1
        elif label in ['CARDINAL']: cardinality += 1
        
    return specific_keyword_count, numeric_keyword_count, contextual_keyword_count, cardinality

In [116]:
sentsdf['SKC'],sentsdf['NKC'],sentsdf['CKC'],sentsdf['Cardinality'] = zip(*sentsdf['Sentence'].apply(countKeywords))
sentsdf

,Sentence,SKC,NKC,CKC,Cardinality
0,"Potential strategies for combating COVID – 19 Abstract Coronavirus disease 2019, also known as COVID – 19, is caused by a novel coronavirus named severe acute respiratory syndrome coronavirus 2, or SARS – CoV – 2.",0,0,1,3
1,"The infection has now catapulted into a full – blown pandemic across the world, which has affected more than 2 million people and has led to approximately 150, 000 fatalities all over the world (WHO).",1,0,0,3
2,"In this review, we elaborate all currently available data that shed light on possible methods for treatment of COVID – 19, such as antiviral drugs, corticosteroids, convalescent plasma, and potentially effective vaccines.",0,1,0,1
3,"Additionally, ongoing and discontinued clinical trials that have been carried out for validating probable treatments for COVID – 19 are discussed.",0,0,0,1
4,The review also elaborates the prospective approach and the possible advantages of using convalescent plasma and stem cells for the improvement of clinical symptoms and meeting the demand for an instantaneous cure.,0,1,0,0
...,...,...,...,...,...
469,"Although extensive research is being conducted to develop a suitable vaccine against COVID – 19, there is a dire need to shift a major part of the ongoing research towards the treatment of pneumonia in patients, which is often fatal.",0,0,0,1
470,The astounding data regarding mesenchymal stem cells offers a hopeful approach to the use of an endogenous pathway for the treatment of disease.,0,0,0,0
471,"In the case of pneumonia caused by COVID – 19, the balance between the antiviral response and the regulation of LIF action against a cytokine storm may be lost, which can reduce the overall beneficial effect of therapy.",0,0,1,0
472,"Nevertheless, the study findings indicating the safe and effective use of mesenchymal stem cell therapy in treating COVID – 19 are indeed remarkable and present a modern approach to the efficient and safe treatment of critically ill patients.",0,0,0,1


In [117]:
'''
Displays the text surrounding the sentence with the provided index. To refer to the context, perhaps.
args: idx - index of sentence in question.
    df = Dataframe, default is sentsdf.
    pm = plus or minus for the indices of surrounding sentences.
'''
def context(idx, pm = 1, df = sentsdf):
    n = len(df)
    
    if (idx-pm) < 0 and (idx+pm) > n:
        return df
    elif (idx-pm) < 0:
        return df.loc[0:idx+pm,:]
    elif (idx-pm) > n:
        return df.loc[idx-pm:n,:]
    return df.loc[idx-pm:idx+pm,:]

from collections import Counter

'''
Print entity labels and their occurences within a given sentence.
args: sentence - The sentence in question.
    from_keywords - Whether or not to only include info on labels in the contextual keywords. Does not by default.
return: list containing tuples of entity labels and their occurences.
'''
def printEntityLabels(sentence, from_keywords = False):
    sentence_obj = nlp(sentence)
    ent_labels = [ent.label_ for ent in sentence_obj.ents if (not from_keywords or ent.label_ in contextual_keywords)]
    labels = Counter(ent_labels).keys()
    counts = Counter(ent_labels).values()
    return list(zip(labels, counts))

'''
Prints all sentences from Dataframe with provided keywords.
args: filter_words: Words to filter results by. By default, the trait keywords.
    df - The Dataframe to filter. By default, sentsdf.
return: Returns Dataframe with only sentences including keywords from the list.
'''
def sentencesWith(filter_words, df=sentsdf):
    if isinstance(filter_words, str):
        filter_regex = filter_words
    elif isinstance(filter_words, list):
        filter_regex = '|'.join(filter_words)
    return df[df.Sentence.str.lower().str.contains(filter_regex)]

In [118]:
import numpy as np

f_out = open("testOutput.txt", "w")

dict_map = {
    # Infectious:
    "infectious":"infectious",
    "infection":"infectious",
    # Contact:
    "contact":"contact",
    # Incubation:
    "incubation":"incubation",
    # Latency:
    "latency":"latency",
    "latent":"latency",
    # Reproduction:
    "reproductive":"reproduction",
    "reproduction":"reproduction",
    "r0":"reproduction",
    # Cases/Deaths
    "fatalities":"deaths",
    "deaths":"deaths",
    "cases":"cases",
    # Transmission
    "transmission":"transmission",
    "transmi*":"transmission",
    # Case-fatality rate:
    "case - fatality":"CFR",
    "case-fatality":"CFR",
    "CFR":"CFR"
}

statsdf = pd.DataFrame([],columns=['Parameter','Estimates','n','Citation','Rule'])
print(statsdf)

from spacy.matcher import Matcher
bigMatcher = Matcher(nlp.vocab)
smallMatcher = Matcher(nlp.vocab)
smallMatcher.add("n_rule",None,[{'TEXT':'n'},{'TEXT':'='},{'LIKE_NUM':True}])
idx = None     

def find_n(doc):
    n_match = smallMatcher(doc)
    if n_match:
        match_id, start, end = n_match[0]
        return int(doc[end-1].text)
    return np.NaN

# takes an NLP doc, a token of the matched estimate, a regex pattern produced by re.compile, and whether or not
# we search bidirectionally (backwards & forwards) or not (just backwards)
def proximitySearch(doc, est_token, regex, bidirectional = False):
    parameter = "None"
    # We start with the first token (index 0):
    closestIdx = abs(0 - est_token.i)
    for token in doc:
        # If the token comes before the estimate and matches a specific keyword:
        if (re.search(regex, token.text.lower())):
            if (((not bidirectional) & (token.i <= est_token.i)) or bidirectional):
                proximity_to_est = abs(token.i - est_token.i)
                # find the closest keyword to our estimate, as long as its within an arbitrary range (i said 15)
                if (proximity_to_est < closestIdx) & (proximity_to_est < 15):
                    closestIdx = token.i
                    parameter = token.text.lower()        
    return parameter

def proximitySearchPrevSentence(doc, regex, idx):
    parameter = "None"
    if (idx > 0):
        prevSent = sents_filt.loc[idx - 1].Sentence
        prevdoc = nlp(prevSent)
        closestIdx = 0
        for token in prevdoc:
            if (re.search(regex, token.text.lower())):
                    if (token.i > closestIdx):
                        closestIdx = token.i
                        parameter = token.text.lower()
    return parameter
    
def funnel_values(estimates, parameter, rule_name, doc):
    global statsdf
    
    n = find_n(doc) #could make this faster
    # Parameter IS in dictionary
    if parameter:
        # Checking if estimates already included, in which case doesn't add a new row
        if not ((statsdf['Parameter'] == parameter) &\
             (statsdf['Estimates'] == estimates)).any():
            new_row = {'Parameter':parameter,\
                       'Estimates':estimates,\
                       'n':n,\
                       'Citation':idx,\
                       'Rule':rule_name}
            statsdf = statsdf.append(new_row,ignore_index=True)
         
    # Parameter is NOT in dictionary
    elif not (idx in list(statsdf.Citation) and\
        estimates in list(statsdf.Estimates.loc[statsdf.Citation == idx])):
        statsdf.loc[len(statsdf)] = [parameter, estimates, n, idx, rule_name]
        
def pm_map(matcher, doc, id, matches):
    for match_id, start, end in matches:
        string = str(doc[start:end])
        split_span = string.split()
        
        # Get the rule
        rule = nlp.vocab.strings[match_id]
        
        # Get our leftmost estimate token for search:
        avg_token = doc[end-3]
        
        # Compute interval
        moe = round(float(doc[end-1].text),2)
        avg = round(float(avg_token.text),2)
        estimates = (avg-moe,avg+moe)
        
        # Search for corresponding parameter
        parameter = proximitySearch(doc, avg_token, specReg)
        
        # Failed? Try searching last sentence:
        if (parameter == "None"):
            parameter = proximitySearchPrevSentence(doc, specReg, idx)
            rule = rule + " (prev sentence)"
        
        funnel_values(estimates, dict_map.get(parameter), rule, doc)

# days_map finds matches for "# days", then searches the rest of the sentences' tokens for specific keywords to
# store as the corresponding parameter.
def num_map(matcher, doc, id, matches):
    for match_id, start, end in matches: 
        rule = nlp.vocab.strings[match_id]
        currSent = doc.text
        sentIdx = sents_filt.index[sents_filt.Sentence == currSent]
        est_token = doc[end-2]
        estimate = est_token.text
        
        # Search for corresponding parameter:
        parameter = proximitySearch(doc, est_token, specReg)
        
        # Failed? Try searching last sentence:
        if (parameter == "None"):
            parameter = proximitySearchPrevSentence(doc, specReg, idx)
            rule = rule + " (prev sentence)"
        
        funnel_values(estimate, dict_map.get(parameter), rule, doc)
        
# and_map works like days map, but for ranges specified with "and"
def and_map(matcher, doc, id, matches):
    for match_id, start, end in matches:
        rule = nlp.vocab.strings[match_id]
        # Find leftmost token (for searching):
        lower_est_token = doc[start]
        
        # Compute interval:
        lower = round(float(lower_est_token.text),2)
        upper = round(float(doc[end-1].text),2)
        estimates = (lower,upper)
        
        # Search for corresponding parameter:
        parameter = proximitySearch(doc, lower_est_token, specReg)
        
        # Failed? Try searching last sentence:
        if (parameter == "None"):
            parameter = proximitySearchPrevSentence(doc, specReg, idx)
            rule = rule + " (prev sentence)"
            
        funnel_values(estimates, dict_map.get(parameter), rule, doc)
        
def cases_deaths_map(matcher, doc, id, matches):
    for match_id, start, end in matches:
        estimates = int(doc[start].text.replace(",",""))
        parameter = doc[end-1].text.replace("fatalities","deaths")
        funnel_values(estimates, parameter, "cases_deaths_map", doc)
        
def r0_map(matcher, doc, id, matches):
    for match_id, start, end in matches:
        funnel_values(round(float(doc[end-1].text),2),"reproduction","r0_map",doc)

Empty DataFrame
Columns: [Parameter, Estimates, n, Citation, Rule]
Index: []


In [119]:
bigMatcher.add("pm_rule", pm_map, [{"IS_ALPHA":True},{"IS_ALPHA":True},{"IS_ALPHA":True},\
           {"LIKE_NUM":True}, {"TEXT":"±"}, {"LIKE_NUM":True}])
bigMatcher.add("and_rule", and_map,\
               [{"IS_DIGIT":True}, {"TEXT":"and"}, {"IS_DIGIT":True}],\
               [{"IS_DIGIT":True}, {"TEXT":"–"}, {"IS_DIGIT":True}],\
               [{"IS_DIGIT":True}, {"TEXT":"to"}, {"IS_DIGIT":True}])
bigMatcher.add("num_rule", num_map,\
               [{"IS_ALPHA":True},{"IS_ALPHA":True},{"LIKE_NUM":True},{"TEXT":{"REGEX":"day*|%"}}],\
               [{"TEXT":{"REGEX":"was|is"}},{"LIKE_NUM":True},{"TEXT":{"REGEX":"day*|%"}}])
bigMatcher.add("cases_deaths_rule", cases_deaths_map, 
               [{"LIKE_NUM":True},{"TEXT":{"REGEX":"fatalities|deaths|cases"}}])
bigMatcher.add("r0_map",r0_map,[{"LOWER":"r0"},{"TEXT":"="},{"LIKE_NUM":True}])

In [134]:
statsdf = statsdf.iloc[0:0]

sents_filt = sentsdf
for idx in sents_filt.index:
    sentence = sents_filt.Sentence[idx]
    doc = nlp(sentence)
    matches = bigMatcher(doc)

None_indices =  [i for i, x in enumerate(list(statsdf.Parameter)) if x == None]
non_None_indices = list(set(list(statsdf.index))^set(None_indices))

statsdf_without_None = statsdf.iloc[non_None_indices]
statsdf_with_None = statsdf.iloc[None_indices]

statsdf = pd.concat([statsdf_without_None, statsdf_with_None])
statsdf

,Parameter,Estimates,n,Citation,Rule
0,deaths,0,NaN,1,cases_deaths_map
6,incubation,"(1.0, 14.0)",NaN,46,and_rule
7,incubation,5,NaN,50,num_rule
8,infectious,7,NaN,51,num_rule
9,transmission,7,NaN,51,num_rule
10,reproduction,"(2.0, 8.0)",NaN,52,and_rule
11,incubation,"(5.0, 12.0)",NaN,54,and_rule (prev sentence)
12,incubation,"(7.0, 8.0)",NaN,54,and_rule (prev sentence)
13,infectious,nine,NaN,55,num_rule (prev sentence)
21,cases,19,NaN,194,cases_deaths_map


In [150]:
mostSKCdf = sentsdf.sort_values(by="SKC", ascending=False).head(20)
matchesdf = sentsdf.iloc[list(set(statsdf.Citation))].sort_index(inplace=False)
relevantdf_indices = list(set(mostSKCdf.index)-set(matchesdf.index))
relevantdf = pd.concat([matchesdf, sentsdf.iloc[relevantdf_indices].sort_index(inplace=False)])
relevantdf

,Sentence,SKC,NKC,CKC,Cardinality
1,"The infection has now catapulted into a full – blown pandemic across the world, which has affected more than 2 million people and has led to approximately 150, 000 fatalities all over the world (WHO).",1,0,0,3
11,other betacoronaviruses (β – CoVs) have been were found to be closely related (more than 90% sequence identity) to a CoV strain found in bats.,0,0,0,0
27,"UV light is known to be effective against CoVs, as are organic solvents such as ether, alcohol (60 – 95% ), chloroform, and disinfectants with a high concentration of chlorine.",0,0,0,1
33,"The epidemics of SARS – CoV and MERS – CoV in 2002 – 2003 and 2012, respectively, were caused by CoVs that caused severe respiratory diseases in humans.",1,0,3,0
41,"The SARS outbreak severely affected China and neighboring countries, with a fatality rate of about 9% , while the MERS outbreak affected Asia with symptoms similar to those of SARS.",2,0,2,0
46,"The virus has an incubation period of 1 – 14 in humans, depending on the conditions and the individual.",1,1,0,2
50,"The characteristics of the infection and transmission are given below:In the early days of the outbreak, when cases were restricted only to China, the mean incubation period of SARS – CoV – 2 was reported to be about 5 days.",4,2,3,1
51,"The time on average for its exponential growth (and infection of other individuals) was observed to be about 7 days, while the mean interval for transmission from person to person was also about 7 days.",2,2,2,0
52,The R0 value was found to be between 2 and 8.,0,1,1,1
54,"Patients with mild symptoms reported the mean interval to be approximately 5 and 12 days from the beginning of infection to their first treatment and hospital care, respectively, and in patients with severe symptoms, the interval was 7 and 8 days, respectively.",1,1,2,0
